## Silverbox

Silverbox is the name of a nonlinear system identification benchmark, proposed in 2004. Data, baselines and more info can be found at http://nonlinearbenchmark.org/#Silverbox.

State-space formulation of Silverbox's dynamics:

$$\begin{align}
\mu \frac{d^2 x(t)}{dt^2} + \nu \frac{d x(t)}{dt} + \kappa(x(t)) x(t) =&\ u(t) + w(t) \\
\kappa(x(t)) =&\ \alpha + \beta x^2(t) \\
y(t) =&\ x(t) + e(t)
\end{align}$$

where
$$\begin{align}
\mu     =&\ \text{mass} \\
\nu     =&\ \text{viscous damping} \\
\kappa(x(t)) =&\ \text{nonlinear spring} \\
y(t)    =&\ \text{observation (displacement)} \\
x(t)    =&\ \text{state (displacement)} \\
u(t)    =&\ \text{force} \\
e(t)    =&\ \text{measurement noise} \\
w(t)    =&\ \text{process noise}
\end{align}$$

### Steps to solve

I now take a series of steps to re-write this problem:

#### 1. Assume constant spring coefficient κ

$$ \mu \frac{d^2 x(t)}{dt^2} + \nu \frac{d x(t)}{dt} + κ x(t) = u(t) + w(t)$$

#### 2. Divide by leading coefficient

$$ \frac{d^2 x(t)}{dt^2} + \frac{\nu}{\mu} \frac{d x(t)}{dt} + \frac{κ}{\mu} x(t) = \frac{1}{\mu} u(t) + \frac{1}{\mu} w(t)$$

#### 3. Substitute standard variables

$$ \frac{d^2 x(t)}{dt^2} + 2\zeta \omega_0 \frac{d x(t)}{dt} + \omega_0^2 x(t) - \frac{u(t)}{\mu} = \frac{w(t)}{\mu}$$

where $$\begin{align} 
\zeta =&\ \frac{\nu}{2\sqrt{\mu \kappa}} \\ 
\omega_0 =&\ \sqrt{\frac{\kappa}{\mu}} \, .
\end{align}$$

#### 4. Apply Euler's method to obtain difference equation (step size is 1)

-> Forward Euler:

$$\begin{align}
\frac{x(t+2h)-2x(t+h)+x(t)}{h^2} + 2\zeta \omega_0 \frac{x(t+h)-x(t)}{h} + \omega_0^2 x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} \\
x(t+2) + 2(\zeta \omega_0 - 1) x(t+1) + (1 - 2 \zeta \omega_0 + \omega_0^2) x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} 
\end{align}$$

-> Backward Euler:

$$\begin{align}
\frac{x(t)-2x(t-h)+x(t-2h)}{h^2} - 2\zeta \omega_0 \frac{x(t)-x(t-h)}{h} - \omega_0^2 x(t) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu}  \\
(1 + 2 \zeta \omega_0 + \omega_0^2)x(t) - 2(1 + \zeta \omega_0)x(t-1) - x(t-2) - \frac{u(t)}{\mu} =&\ \frac{w(t)}{\mu} \\
x(t) - \frac{2(1 + \zeta \omega_0)}{(1 + 2\zeta \omega_0 + \omega_0^2)}x(t-1) - \frac{1}{(1 + 2\zeta \omega_0 + \omega_0^2)}x(t-2) - \frac{u(t)}{\mu(1 + 2\zeta \omega_0 + \omega_0^2)} =&\ \frac{w(t)}{\mu(1 + 2\zeta \omega_0 + \omega_0^2)}
\end{align}$$
    
Change to shorthand notation:

$$x_t - \alpha x_{t-1} - \beta x_{t-2} - \gamma u_t = \gamma w_t$$

where 
$$\begin{align} 
\alpha =&\ \frac{2(1 + \zeta \omega_0)}{1 + 2\zeta \omega_0 + \omega_0^2} \\
\beta =&\ \frac{1}{1 + 2\zeta \omega_0 + \omega_0^2} \\
\gamma =&\ \frac{1}{\mu(1 + 2 \zeta \omega_0 + \omega_0^2)}
\end{align}$$

#### 5. Convert to multivariate first-order difference form

Stick to backward Euler (matches AR structure)
- Backward Euler:

    $$z_t = M z_{t-1} + N u_t + N w_t$$

    where $z_t = [x_t\ \ x_{t-1}]$, $M = [α \ \ β; 1\ \ 0]$, $N = [γ\ \ 0]$

#### 6. Convert to Gaussian probability

- Backward Euler:

$$z_t \sim \mathcal{N}(M z_{t-1} + N u_t, N \tau)$$

where $w_t \sim \mathcal{N}(0, \tau)$

#### 7. Observation likelihood

$$y_t \sim \mathcal{N}(c z_t, σ)$$

where $e_t \sim \mathcal{N}(0, \sigma)$, $c = [1\ \ 0]$

Now, I need priors for $\alpha$, $\beta$, $\gamma$, $\tau$, $\sigma$. Given three equations and three unknowns, I can recover $\zeta$, $\omega_0$ and $\mu$ from $\alpha$, $\beta$, and $\gamma$. The variables are all strictly positive, which means they can be modeled by gamma distributions:

$$\begin{align}
\alpha \sim&\ Γ(1, 1e3) \\
\beta \sim&\ Γ(1, 1e3) \\
\gamma \sim&\ Γ(1, 1e3) \\
\tau \sim&\ Γ(1, 1e3) \\
\sigma \sim&\ Γ(1, 1e3) 
\end{align}$$

--> Implementation with ForneyLab and AR node

In [1]:
# Generate time-series
using CSV
using DataFrames
using Plots

# Read data from CSV file
df = CSV.read("../data/SNLS80mV.csv", ignoreemptylines=true)
df = select(df, [:V1, :V2])

# Shorthand
input = df[:,1]
output = df[:,2]

# Time horizon
T = size(df, 1)

131072

In [2]:
using ForneyLab
using LAR
using LAR.Node, LAR.Data
using ProgressMeter

┌ Info: Precompiling LAR [c3bc7fac-5998-4d64-8961-b7df36e0e4ce]
└ @ Base loading.jl:1273


I will introduce another shorthand: $\theta = [\alpha\ \ \beta]$.

In [14]:
# Start graph
g = FactorGraph()

# Static parameters
@RV log_θ ~ GaussianMeanPrecision(placeholder(:m_θ_t, dims=(2,)), placeholder(:w_θ_t, dims=(2,2)))
# @RV γ ~ Gamma(1, 1e3)
@RV τ ~ Gamma(placeholder(:a_τ_t, dims=(1,)), placeholder(:b_τ_t, dims=(1,)))

# Nonlinear function
f(x) = exp.(x)

# Nonlinear node
@RV θ ~ Nonlinear{Sampling}(log_θ, f, n_samples=100)

# I'm fixing measurement noise σ
σ = 10.

# Observation selection variable
c = [1, 0]

# State prior
@RV z_t ~ GaussianMeanPrecision(placeholder(:m_z_t, dims=(2,)), placeholder(:w_z_t, dims=(2, 2)))

# Autoregressive node
@RV x_t ~ Autoregressive(θ, z_t, τ)

# Specify likelihood
@RV y_t ~ GaussianMeanPrecision(dot(c, x_t), σ)

# Placeholder for data
placeholder(y_t, :y_t)

# Draw time-slice subgraph
ForneyLab.draw(g)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 1808332224511346075 -->
 
 1808332224511346075 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 17956468478151218775 -->
 
 17956468478151218775 
 
 placeholder_m_θ_t 
 
<!-- 1808332224511346075--17956468478151218775 -->
 
 1808332224511346075--17956468478151218775 
 
 m_θ_t 
 1 out 
 2 m 
 
<!-- 1347366131856641983 -->
 
 1347366131856641983 
 
 placeholder_w_θ_t 
 
<!-- 1808332224511346075--1347366131856641983 -->
 
 1808332224511346075--1347366131856641983 
 
 w_θ_t 
 1 out 
 3 w 
 
<!-- 9768678675354884993 -->
 
 9768678675354884993 
 
 placeholder_a_τ_t 
 
<!-- 7009322650665851659 -->
 
 7009322650665851659 
 
 𝒩 
 gaussianmeanprecision_3 
 
<!-- 4210938925998463944 -->
 
 4210938925998463944 
 
 clamp_2 
 
<!-- 7009322650665851659--4210938925998463944 -->
 
 7009322650665851659--4210938925998463944 
 
 clamp_2 
 1 out 
 3 w 
 
<!-- 6281202576269760228 -->
 
 6281202576269760228 
 
 dot 
 dotproduct_1 
 
<!-- 7009322650665851659--6281202576269760228 -->
 
 7009322650665851659--6281202576269760228 
 
 variable_1 
 1 out 
 2 m 
 
<!-- 17970856126810498454 -->
 
 17970856126810498454 
 
 placeholder_m_z_t 
 
<!-- 15613601380873131219 -->
 
 15613601380873131219 
 
 placeholder_y_t 
 
<!-- 15613601380873131219--7009322650665851659 -->
 
 15613601380873131219--7009322650665851659 
 
 y_t 
 1 out 
 1 out 
 
<!-- 6995987696338503631 -->
 
 6995987696338503631 
 
 AR 
 autoregressive_1 
 
<!-- 16442135598161508209 -->
 
 16442135598161508209 
 
 Nonlinear{Sampling} 
 nonlinear_1 
 
<!-- 6995987696338503631--16442135598161508209 -->
 
 6995987696338503631--16442135598161508209 
 
 θ 
 1 out 
 3 θ 
 
<!-- 5289366695121826167 -->
 
 5289366695121826167 
 
 Gam 
 gamma_1 
 
<!-- 6995987696338503631--5289366695121826167 -->
 
 6995987696338503631--5289366695121826167 
 
 τ 
 1 out 
 4 γ 
 
<!-- 5706829205732008425 -->
 
 5706829205732008425 
 
 𝒩 
 gaussianmeanprecision_2 
 
<!-- 6995987696338503631--5706829205732008425 -->
 
 6995987696338503631--5706829205732008425 
 
 z_t 
 1 out 
 2 x 
 
<!-- 4841254387339933449 -->
 
 4841254387339933449 
 
 placeholder_b_τ_t 
 
<!-- 15336331892116254960 -->
 
 15336331892116254960 
 
 clamp_1 
 
<!-- 16442135598161508209--1808332224511346075 -->
 
 16442135598161508209--1808332224511346075 
 
 log_θ 
 1 out 
 2 in1 
 
<!-- 5289366695121826167--9768678675354884993 -->
 
 5289366695121826167--9768678675354884993 
 
 a_τ_t 
 1 out 
 2 a 
 
<!-- 5289366695121826167--4841254387339933449 -->
 
 5289366695121826167--4841254387339933449 
 
 b_τ_t 
 1 out 
 3 b 
 
<!-- 5706829205732008425--17970856126810498454 -->
 
 5706829205732008425--17970856126810498454 
 
 m_z_t 
 1 out 
 2 m 
 
<!-- 9584320092438205722 -->
 
 9584320092438205722 
 
 placeholder_w_z_t 
 
<!-- 5706829205732008425--9584320092438205722 -->
 
 5706829205732008425--9584320092438205722 
 
 w_z_t 
 1 out 
 3 w 
 
<!-- 6281202576269760228--6995987696338503631 -->
 
 6281202576269760228--6995987696338503631 
 
 x_t 
 1 y 
 2 in1 
 
<!-- 6281202576269760228--15336331892116254960 -->
 
 6281202576269760228--15336331892116254960 
 
 clamp_1 
 1 out 
 3 in2

In [15]:
# Infer an algorithm
q = PosteriorFactorization(z_t, x_t, θ, τ, ids=[:z, :x, :θ, :τ])
algo = variationalAlgorithm(q, free_energy=true)
source_code = algorithmSourceCode(algo, free_energy=true)
eval(Meta.parse(source_code));
# println(source_code);

In [16]:
# Inference parameters
num_iterations = 10

# Initialize marginal distribution and observed data dictionaries
data = Dict()
marginals = Dict()

# Initialize arrays of parameterizations
params_m_x = zeros(T+1,1)
params_w_x = zeros(T+1,1)
params_m_z = zeros(T+1,2)
params_w_z = zeros(T+1,2,2)
params_m_θ = zeros(T+1,2)
params_w_θ = zeros(T+1,2,2) 
params_τ = zeros(T+1,2)

# Start progress bar
p = Progress(T, 1, "At time ")

# Perform inference at each time-step
for t = 1:T

    # Update progress bar
    update!(p, t)

    # Initialize marginals
    marginals[:x_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_m_x[t,1], w=params_w_x[t,1])
    marginals[:z_t] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_m_z[t,:], w=params_w_z[t,:,:])
    marginals[:θ] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=params_m_θ[t,:], w=params_w_θ[t,:,:])
    marginals[:τ] = ProbabilityDistribution(Univariate, Gamma, a=params_τ[t,1], b=params_τ[t,2])
    
    data = Dict(:y_t => output[t],
                :m_z_t => params_m_z[t,:],
                :w_z_t => params_w_z[t,:,:],
                :m_θ_t => params_m_θ[t,:],
                :w_θ_t => params_w_θ[t,:,:],
                :a_τ_t => params_τ[t,1],
                :b_τ_t => params_τ[t,2])

    # Iterate variational parameter updates
    for i = 1:num_iterations

        stepx!(data, marginals)
        stepz!(data, marginals)
        stepθ!(data, marginals)
        stepτ!(data, marginals)
    end

    # Store current parameterizations of marginals
    params_m_x[t+1,1] = unSafeMean(marginals[:x_t])
    params_w_x[t+1,1] = unSafePrecision(marginals[:x_t])
    params_m_z[t+1,:] = unSafeMean(marginals[:z_t])
    params_w_z[t+1,:,:] = unSafePrecision(marginals[:z_t])
    params_m_θ[t+1,:] = unSafeMean(marginals[:θ])
    params_w_θ[t+1,:,:] = unSafePrecision(marginals[:θ])
    params_τ[t+1,1] = marginals[:τ].params[:a]
    params_τ[t+1,2] = marginals[:τ].params[:b]

end

MethodError: MethodError: no method matching prod!(::ProbabilityDistribution{Multivariate,GaussianMeanPrecision}, ::ProbabilityDistribution{Univariate,Function})
Closest candidates are:
  prod!(!Matched::ProbabilityDistribution{Univariate,SampleList}, ::ProbabilityDistribution{Univariate,family} where family<:Union{Function, FactorNode}) at /home/wmkouw/.julia/dev/ForneyLab/src/factor_nodes/sample_list.jl:82
  prod!(!Matched::ProbabilityDistribution{Univariate,SampleList}, ::ProbabilityDistribution{Univariate,family} where family<:Union{Function, FactorNode}, !Matched::ProbabilityDistribution{Univariate,SampleList}) at /home/wmkouw/.julia/dev/ForneyLab/src/factor_nodes/sample_list.jl:82
  prod!(!Matched::ProbabilityDistribution{Univariate,Function}, ::ProbabilityDistribution{Univariate,Function}) at /home/wmkouw/.julia/dev/ForneyLab/src/engines/julia/update_rules/nonlinear.jl:569
  ...